In [1]:
import pandas as pd

In [2]:
df1=pd.read_csv('final.csv')
df1

,Unnamed: 0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13,class_encoded
0,0,-596.27124,105.507520,-16.348253,34.213352,-7.040627,9.673249,15.672894,-18.594084,12.680883,-3.053727,-9.320203,5.608082,-5.410851,0
1,1,-359.78244,72.187680,27.768223,38.328430,5.198799,8.380541,10.607283,-13.184477,11.505762,-4.186356,-6.031159,2.438568,-6.354579,0
2,2,-424.57343,57.615242,12.128860,49.243443,-2.526363,11.702657,11.698957,-2.770733,-1.388484,-1.123289,-11.638446,14.478795,-10.840614,0
3,3,-524.17444,103.392075,-7.327183,56.391010,10.343275,4.731972,23.960910,-0.654973,6.595408,4.164212,-6.080491,7.985336,0.112556,0
4,4,-519.28940,101.250860,20.647596,29.956661,8.789474,8.442513,3.100049,5.523725,0.429982,5.303805,-0.014299,4.767339,0.963716,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,63,-473.95490,94.092310,20.465342,14.455287,0.151350,3.607172,-2.468565,-5.190783,0.989746,-2.069601,-10.654523,0.043760,2.364469,3
64,64,-569.43933,38.809494,15.469359,35.134655,-19.263390,-2.875730,-5.993804,-12.542236,-8.285859,-0.797593,-6.813976,-10.226942,-7.345919,3
65,65,-534.11633,45.754597,6.438214,24.977623,1.711067,9.408805,0.141316,-0.821431,-1.418567,3.832415,-2.813458,-1.262796,-4.410855,3
66,66,-552.32140,65.580140,21.206110,12.841641,-0.550191,12.242992,-0.532321,-1.793944,2.008286,-4.199604,-3.661440,-2.616800,-4.645094,3


In [3]:
df1=df1.drop(columns=['Unnamed: 0'])

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
X = df1.iloc[:, :-1] 
y = df1.iloc[:, -1]   
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42  # For reproducibility
)

train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print("Training set distribution:")
print(train_df[y.name].value_counts(normalize=True))

print("\nTest set distribution:")
print(test_df[y.name].value_counts(normalize=True))

Training set distribution:
class_encoded
1    0.259259
3    0.259259
2    0.240741
0    0.240741
Name: proportion, dtype: float64

Test set distribution:
class_encoded
0    0.285714
2    0.285714
3    0.214286
1    0.214286
Name: proportion, dtype: float64


In [5]:
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
import keras_tuner as kt

In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from kerastuner import RandomSearch
from keras.utils import to_categorical

# Assuming X_train, X_test, y_train, and y_test are already defined

# One-hot encoding the labels
num_classes = 4
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

def build_model(hp):
    model = Sequential()
    num_layers = hp.Int('num_layers', 4, 10)

    # Input layer
    input_dim = X_train.shape[1]  # Update to your input feature dimension
    model.add(Dense(hp.Int('input_units', 32, 128, step=16), 
                    activation='relu', 
                    input_shape=(input_dim,)))
    model.add(BatchNormalization())

    for i in range(num_layers - 1):
        num_units = hp.Int(f'dense_units_{i}', 32, 128, step=16)
        model.add(Dense(num_units, activation=hp.Choice(f'activation_{i}', ['relu', 'tanh', 'sigmoid']),
                        kernel_regularizer='l2'))  # Adding L2 regularization
        model.add(BatchNormalization())
        
        dropout_rate = hp.Float(f'dropout_rate_{i}', 0.0, 0.5, step=0.1)
        model.add(Dropout(dropout_rate))

    model.add(Dense(num_classes, activation='softmax'))

    optimizer_options = ['adam', 'sgd', 'rmsprop']
    model.compile(loss='categorical_crossentropy', 
                  optimizer=hp.Choice('optimizer', optimizer_options), 
                  metrics=['accuracy'])
    
    return model

# Initializing the tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=20,
    project_name='ann_tuning'
)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Adding batch size hyperparameter
batch_size = 8

# Start the tuning process
tuner.search(X_train, y_train, 
             epochs=50,
             batch_size=batch_size,  # Use the batch size hyperparameter
             validation_data=(X_test, y_test),
             callbacks=[early_stopping])

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
tuner.results_summary()


Trial 20 Complete [00h 00m 12s]
val_loss: 3.3864762783050537

Best val_loss So Far: 2.8620076179504395
Total elapsed time: 00h 03m 50s
Results summary
Results in .\ann_tuning
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 16 summary
Hyperparameters:
num_layers: 4
input_units: 48
dense_units_0: 112
activation_0: sigmoid
dropout_rate_0: 0.1
dense_units_1: 48
activation_1: tanh
dropout_rate_1: 0.1
dense_units_2: 80
activation_2: sigmoid
dropout_rate_2: 0.4
optimizer: adam
dense_units_3: 96
activation_3: sigmoid
dropout_rate_3: 0.1
dense_units_4: 32
activation_4: relu
dropout_rate_4: 0.2
dense_units_5: 128
activation_5: relu
dropout_rate_5: 0.2
dense_units_6: 112
activation_6: sigmoid
dropout_rate_6: 0.30000000000000004
dense_units_7: 32
activation_7: tanh
dropout_rate_7: 0.30000000000000004
dense_units_8: 80
activation_8: tanh
dropout_rate_8: 0.4
Score: 2.8620076179504395

Trial 00 summary
Hyperparameters:
num_layers: 4
input_units: 96
dense_units_0: 128
activat

In [8]:
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 48)                672       
                                                                 
 batch_normalization (BatchN  (None, 48)               192       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 112)               5488      
                                                                 
 batch_normalization_1 (Batc  (None, 112)              448       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 112)               0         
                                                                 
 dense_2 (Dense)             (None, 48)                5

In [9]:
test_loss, test_accuracy = best_model.evaluate(X_test, y_test, verbose=0)

# Print the test accuracy
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Test Loss: 2.8620
Test Accuracy: 0.2857


In [10]:
best_model.save('ann.h5')